In [1]:
import requests
from bs4 import BeautifulSoup
import os
import fitz
import io
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import faiss
import numpy as np
import pandas as pd
import time
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

/home/aganap12/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-16 16:20:57.278690: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 16:21:03.675476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [48]:
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 7            |        cudaMalloc retries: 8         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   4541 MiB |  22876 MiB |   8497 GiB |   8492 GiB |\n|       from large pool |   4435 MiB |  22765 MiB |   8448 GiB |   8444 GiB |\n|       from small pool |    106 MiB |    112 MiB |     48 GiB |     48 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   4541 MiB |  22876 MiB |   8497 GiB |   8492 GiB |\n|       from large pool |   4435 MiB |  22765 MiB |

In [2]:
def parse_base_html():
    base_url = "https://proceedings.neurips.cc/paper_files/paper/2023"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paper_li = soup.find_all('li', class_='conference')
    paper_links = []
    for i in range(len(paper_li)):
        paper_a = paper_li[i].find('a')
        paper_links.append(paper_a.get('href'))
    final_urls = modify_links(paper_links)
    return final_urls

In [3]:
def modify_links(paper_links):
    paper_urls = []
    for link in paper_links:
        paper_url = "https://proceedings.neurips.cc" + \
                    link.replace("-Abstract-", "-Paper-")\
                        .replace("/hash/", "/file/")\
                        .replace(".html", ".pdf")
        paper_urls.append(paper_url)
    return paper_urls

In [4]:
def extract_text_from_pdf(paper_url):
    response = requests.get(paper_url)
    pdf_content = response.content
    text = ""
    with fitz.open(stream=io.BytesIO(pdf_content)) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [5]:
def preprocess_text(text):
    tagged_data = []
    return word_tokenize(text.lower())

In [12]:
import chromadb
from replicate.client import Client

replicate = Client(api_token="r8_IbZu0U3qUJ5ZC0TzzMm6xCJ6yU5UxVi16Ejo4")

client = chromadb.PersistentClient(path="./neurips-chromadb-client")
collection = client.get_or_create_collection("neurips2023")

In [13]:
pdf_urls = parse_base_html()
tagged_data = []
i = 1
for url in pdf_urls:
    pdf_text = extract_text_from_pdf(url)
    # preprocessed_text = preprocess_text(pdf_text)
    tagged_data.append(TaggedDocument(words=preprocessed_text, tags=[url]))
    # print(preprocessed_text)
    collection.add(
                documents=pdf_text,
                metadatas={"url": url},
                ids=f"doc{i}",
            )
    i += 1

MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error: cannot find ExtGState resource 'TRP1'
MuPDF error: syntax error

In [14]:
results = collection.query(
    query_texts=["use cases, applications, application areas"],
    n_results=len(pdf_urls),
)

In [15]:
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)
#

llm = HuggingFacePipeline(pipeline=READER_LLM)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 8/8 [00:50<00:00,  6.37s/it]


In [23]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """
                    Using the information contained in the context,
                    give a comprehensive answer to the question.
                    Respond only to the question asked, response should be concise and relevant to the question.
                    Provide response in bullet points with an appropriate title.
                    If the answer cannot be deduced from the context, do not give an answer.
                """,
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>

                    Using the information contained in the context,
                    give a comprehensive answer to the question.
                    Respond only to the question asked, response should be concise and relevant to the question.
                    Provide response in bullet points with an appropriate title.
                    If the answer cannot be deduced from the context, do not give an answer.
                </s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}</s>
<|assistant|>



In [ ]:
user_query = "What are the top use cases or application areas of retrieval augmented generation (RAG)? Emphasize on RAG applications with vector databases as well."
k=len(pdf_urls)
print(f"\nStarting retrieval for {user_query=}...")
print("\n==================================Top k documents==================================")
topk_retrieved_filename =  results['metadatas'][0][:k]
topk_retrieved_file_id =  results['ids'][0][:k]
topk_retrieved_file =  results['documents'][0][:k]
print(f"filename: {topk_retrieved_filename}\nfile_id: {topk_retrieved_file_id}")
print("\n==================================Top document==================================")
retrieved_filename =  results['metadatas'][0][0]
retrieved_file_id =  results['ids'][0][0]
retrieved_file =  results['documents'][0][0]
# print(f"filename: {retrieved_filename['filename']}\nfile_id: {retrieved_file_id}")


In [89]:
def remove_references(text):
    words = text.split()
    try:
        references_index = words.index('References')
        words_before_references = words[:references_index]
        cleaned_text = ' '.join(words_before_references)
    except ValueError:
        return text
    return cleaned_text

In [90]:
retrieved_docs_text = topk_retrieved_file[:100]
for i in range(len(retrieved_docs_text)):
    remove_references(retrieved_docs_text[i])
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)

# print(final_prompt)

In [91]:
from langchain_core.output_parsers import JsonOutputParser
answer = llm(final_prompt)
print(answer)

/home/aganap12/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.22 GiB. GPU 0 has a total capacity of 23.49 GiB of which 4.25 GiB is free. Including non-PyTorch memory, this process has 19.23 GiB memory in use. Of the allocated memory 17.38 GiB is allocated by PyTorch, and 1.57 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
# model.build_vocab(tagged_data)
# model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
# document_vectors = {}
# for url in pdf_urls:
#     pdf_text = extract_text_from_pdf(url)
#     preprocessed_text = preprocess_text(pdf_text)
#     vector_representation = model.infer_vector(preprocessed_text)
#     document_vectors[url] = vector_representation
#     print("Added: ", url)

In [ ]:
# vector_dim = len(next(iter(document_vectors.values())))
# print(vector_dim)

In [ ]:
# index = faiss.IndexFlatL2(vector_dim)
# vectors_np = np.array(list(document_vectors.values())).astype('float32')
# query_text = "use cases, applications, application areas"
# query_vector = model.infer_vector(query_text.lower().split(" "))
# query_vector_np = np.array([query_vector]).astype('float32')
# k = len(pdf_urls)

# index.reset()
# index.add(vectors_np)
# start_time = time.time()
# distances, indices = index.search(query_vector_np, k=k)
# end_time = time.time()
# retrieval_time = (end_time - start_time) * 1e6
# print("Retrieval time:", retrieval_time, "microseconds")

# results = pd.DataFrame({'distances': distances[0], 'ann': indices[0]})
# print(results.head())